# <div align="center">Perceptron Demo</div>
### <div align="center">*Author: Brendan Schlaman*</div>

In [ ]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
from mltools.utils.math.vectors import Vector
from matplotlib.patches import Circle
import random
from bpyutils.formatting.std import data_print
import time

---

## 1) Reflection about origin

This plot shows that finding a dividing hyperplane
is equivalent to finding a hyperplane (through the origin)
with all points on one side after reflection of the
negative points about the origin.

In [ ]:
X0 = np.array([(2,3),(3,2),(3,4),(1,1),(1,3)] ) # label -1
X1 = np.array([(2,.5),(-2,2),(-1,1),(0,.5),(-1,-1)]   ) # label 1
# points are projected onto (1, y, z) plane
X0 = np.hstack((np.ones((len(X0), 1)), X0))
X1 = np.hstack((np.ones((len(X1), 1)), X1))

# create a common linspace based on the extreme points of x
ls = np.linspace(min(*X0.T[1],*X1.T[1]),max(*X0.T[1],*X1.T[1]), 4)

In [ ]:
# actually run the algorithm to get lines and surfaces to plot
# first, put features and labels into a single datastructure for convenience
Xy = np.concatenate((np.hstack((X0, -np.ones((len(X0))).reshape(-1,1))),np.hstack((X1, np.ones((len(X1))).reshape(-1,1)))))

w = np.zeros((len(Xy.T)-1)) # 3d in this case

def perceptron():
    global w
    misses = 0
    for row in Xy:
        x, y = row[:-1], row[-1]

        if y * w.dot(x) <= 0:
            misses += 1
            w = w + y*x

    return misses

for epoch in range(100):
    if epoch % 10 == 0: print(epoch, end=' ')
    misses = perceptron()
    if misses == 0: break

print(f"\n{w=}")

In [ ]:
def calculate_1yz_projection(w: np.ndarray, ls: np.ndarray) -> tuple:
    slope = -w[1] / w[2]
    offset = -w[0] / w[2]
    return slope * ls + offset

w_1yz_proj = calculate_1yz_projection(w, ls)

In [ ]:
# calculate the hyperplane
# x will be -1 to 1; -1 since I demo reflection about origin
x = np.linspace(-1, 1, 10)
y = np.linspace(ls[0], ls[-1], 10)  # just use the linspace
x, y = np.meshgrid(x, y)
w0, w1, w2 = w
z = (-w0 * x - w1 * y) / w2

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223, projection="3d")
ax4 = fig.add_subplot(224, projection="3d")

ax1.scatter(*X0.T[1:], c="b", marker="o", label="$y_i = -1$")
ax1.scatter(*X1.T[1:], c="r", marker="^", label="$y_i = 1$")
ax1.axhline(0, color="black", linestyle=":")
ax1.axvline(0, color="black", linestyle=":")
ax1.plot(ls, w_1yz_proj, color="darkred", label="hyperplane")
ax1.legend()
ax1.set_title("Data in 2d with hyperplane")

ax2.scatter(*(-X0.T[1:]), c="b", marker="o", label="Class 0")
ax2.scatter(*X1.T[1:], c="r", marker="x", label="Class 1")
ax2.axhline(0, color="black", linestyle=":")
ax2.axvline(0, color="black", linestyle=":")
ax2.set_title(
    "The transformation of the problem after reflection\nof the $-1$ datapoints is not clear in 2d"
)

x00, x01, x02 = X0.T[0], X0.T[1], X0.T[2]
x10, x11, x12 = X1.T[0], X1.T[1], X1.T[2]

ax3.scatter(x00, x01, x02, c="b", marker="o", label="$y_i = -1$")
ax3.scatter(x10, x11, x12, c="r", marker="x", label="$y_i = 1$")
ax3.plot([0, max(*x00, *x10)], [0, 0], [0, 0], "r")
ax3.plot([0, 0], [0, max(*x01, *x11)], [0, 0], "g")
ax3.plot([0, 0], [0, 0], [0, max(*x02, *x12)], "b")
ax3.plot([1, 1], [0, max(*x01, *x11)], [0, 0], "g", linestyle=":", label="y-proj")
ax3.plot([1, 1], [0, 0], [0, max(*x02, *x12)], "b", linestyle=":", label="z-proj")
# this appears off visually because of different axes scale
ax3.quiver(0, 0, 0, w0, w1, w2, normalize=True, color="black", label="$\\vec{w}$")
ax3.plot_surface(x, y, z, alpha=0.4, antialiased=False)
ax3.set_xlabel("x")
ax3.set_ylabel("y")
ax3.set_zlabel("z")
ax3.view_init(azim=225)
ax3.set_ylim(ax3.get_ylim()[::-1])  # flip y axis for more intuitive view
ax3.legend()
ax3.set_title("Data and hyperplane after projection\nonto $(1, y, z)$ plane")

ax4.scatter(-x00, -x01, -x02, c="b", marker="o", label="Class 0")
ax4.scatter(x10, x11, x12, c="r", marker="x", label="Class 1")
ax4.plot([0, max(*x00, *x10)], [0, 0], [0, 0], "r")  # x-axis
ax4.plot([0, 0], [0, max(*x01, *x11)], [0, 0], "g")  # y-axis
ax4.plot([0, 0], [0, 0], [0, max(*x02, *x12)], "b")  # z-axis
ax3.plot([1, 1], [0, max(*x01, *x11)], [0, 0], "g", linestyle=":", label="y-proj")
ax3.plot([1, 1], [0, 0], [0, max(*x02, *x12)], "b", linestyle=":", label="z-proj")
ax4.plot_surface(x, y, z, alpha=0.4, antialiased=False)
ax4.set_xlabel("x")
ax4.set_ylabel("y")
ax4.set_zlabel("z")
ax4.view_init(elev=-13, azim=-90)
ax4.set_ylim(ax4.get_ylim()[::-1])  # flip y axis for more intuitive view
ax4.set_title(
    "$-1$ labeled datapoints reflected w.l.o.g;\nperceptron must find hyperplane with all points on one side"
)


plt.suptitle("Perceptron equivalence after reflection of one of the labels", fontweight='bold')
plt.show()

---

## 2) Demo of algorithm

1. Run the below cell
1. Place points on the canvas
1. Run the next cell and watch the canvas update

In [ ]:
class FeaturePlotEventHandler:
    def __init__(self, event_modifier_key):
        self.event_modifier_key = event_modifier_key
        # there may be better alternatives to this class persisting the features
        self.features = []

    def __call__(self, event):
        if event.key != self.event_modifier_key:
            return
        if event.xdata < AX_VIEWPORT[0]: return
        if event.ydata < AX_VIEWPORT[0]: return
        if event.xdata > AX_VIEWPORT[1]: return
        if event.ydata > AX_VIEWPORT[1]: return
        self.features.append((event.xdata, event.ydata))
        render()

# globals
GLOBAL_VECTOR_OFFSET = (0.4, 0.4)
AX_VIEWPORT = (-0.5, 1)
gvo_vec = Vector(*GLOBAL_VECTOR_OFFSET)
# figure, axes, and event handlers
fig, ax = plt.subplots()
ind = np.linspace(*AX_VIEWPORT)  # reusable linear space
plot1 = ax.scatter([], [], color="b", marker="o")
plot2 = ax.scatter([], [], color="r", marker="o")
fpeh1 = FeaturePlotEventHandler(None)
fpeh2 = FeaturePlotEventHandler("shift")
cid1 = fig.canvas.mpl_connect("button_press_event", fpeh1)
cid2 = fig.canvas.mpl_connect("button_press_event", fpeh2)
# w vector
# TODO: make sure to get rid of gvo_vec mention
# quiver = ax.quiver(0, 0, gvo_vec.x, gvo_vec.y, angles="xy", scale_units="xy", scale=1)
adj_quiver = ax.quiver(0, 0, gvo_vec.x, gvo_vec.y, color="black", angles="xy", scale_units="xy", scale=1)
# hyperplane
# (hyperplane_plot,) = ax.plot(ind, ind, ":g")
(adj_hyperplane_plot,) = ax.plot(ind, ind, ":")
# section filling
pos_fill = ax.fill_between(ind, 0, 0, facecolor="b", alpha=0.2)
neg_fill = ax.fill_between(ind, 0, 0, facecolor="r", alpha=0.2)
# test point indicator
circle = Circle((0, 0), 0.03, fill=False, color="black", label="test point")
# text
text = ax.text(AX_VIEWPORT[0] + 0.02, AX_VIEWPORT[0] + 0.02, "")
text2 = ax.text(AX_VIEWPORT[0] + 0.02, AX_VIEWPORT[0] + 0.10*1, "")
text3 = ax.text(AX_VIEWPORT[0] + 0.02, AX_VIEWPORT[0] + 0.10*2, "")
# text4 = ax.text(AX_VIEWPORT[0] + 0.02, AX_VIEWPORT[0] + 0.10*3, "")
# w vector
w = Vector(0, 0)
w3 = Vector(0, 0, 0)


def init_ax():
    # initialize the figure, plots, and feature plotters
    ax.set_xlim(*AX_VIEWPORT)
    ax.set_ylim(*AX_VIEWPORT)
    ax.set_aspect("equal")
    ax.autoscale(False)
    ax.set_title(
        "Perceptron Feature Plotter\n(click to add points, hold shift for red)"
    )
    ax.axhline(y=0, linewidth=0.1)
    ax.axvline(x=0, linewidth=0.1)
    ax.add_patch(circle)

    # create w vector
    global w3
    w3 = Vector(1.8, 1.9, -1.9)
    w3.normalize()
    w3.scale(0.5)

def calculate_normal_line(w: Vector, ind: np.ndarray) -> tuple:
    slope = -w.x / w.y
    offset = -w.z / w.y
    return slope, offset, slope * ind + offset

def render():
    # update w
    # quiver.set_UVC(w.x, w.y)
    adj_quiver.set_UVC(w3.x, w3.y)

    # update features
    if fpeh1.features:
        plot1.set_offsets(fpeh1.features)
    if fpeh2.features:
        plot2.set_offsets(fpeh2.features)

    # update hyperplane
    # w_vector_slope_rot = -w.x / w.y  # swap w.x, w.y; w.x *= -1
    # # dep = w_vector_slope_rot * (ind - gvo_vec.x) + gvo_vec.y
    # dep = w_vector_slope_rot * ind
    # hyperplane_plot.set_data(ind, dep)

    slope, offset, adj_dep = calculate_normal_line(w3, ind)
    adj_hyperplane_plot.set_ydata(adj_dep)

    # update section fills
    global pos_fill
    global neg_fill
    pos_fill.remove()
    neg_fill.remove()
    pos_fill = ax.fill_between(
        ind, adj_dep.tolist(), AX_VIEWPORT[int(w3.y > 0)], facecolor="b", alpha=0.2
    )
    neg_fill = ax.fill_between(
        ind, adj_dep.tolist(), AX_VIEWPORT[int(w3.y < 0)], facecolor="r", alpha=0.2
    )

    # update text
    text.set_text(f"magnitude of w\u20D73: {round(w3.magnitude(), 5)}")
    text2.set_text(f"slope: {slope}")
    text3.set_text(f"offset: {offset}")
    # text4.set_text(f"h/m: {'miss' if miss else 'hit'}")

    # redraw canvas
    ax.figure.canvas.draw()


init_ax()
render()

# claim 1: since hyperplane offset (in y direction w.l.o.g.) depends on w_y,
# w_z can be literally anything because I can find a w_y to compensate


In [ ]:
LABEL_SPACE = (-1, 1)

# data will be of the form ((x, y), label)
data = []
data.extend([(Vector(x, y), LABEL_SPACE[1]) for x, y in plot1.get_offsets()])
data.extend([(Vector(x, y), LABEL_SPACE[0]) for x, y in plot2.get_offsets()])
random.shuffle(data)

data3d = []
data3d.extend([(Vector(x, y, 1), LABEL_SPACE[1]) for x, y in plot1.get_offsets()])
data3d.extend([(Vector(x, y, 1), LABEL_SPACE[0]) for x, y in plot2.get_offsets()])
random.shuffle(data3d)

def perceptron(human: bool) -> int:
    global w3
    misses = 0
    for vec, label in data3d:
        miss = False
        circle.center = vec.x, vec.y # + gvo_vec.x, vec.y + gvo_vec.y
        ax.draw_artist(circle)

        if human:
            input(f"about to test feature: {vec}")
        else:
            time.sleep(0.1)

        if label * w3.dot_product(vec) <= 0:
            misses += 1
            miss = True
        labeled_data = {
            "current w\u20D7": w3,
            "miss count": misses,
            "testing feature": vec,
            "want": label,
            "got": w3.dot_product(vec),
            "result": "miss! adjusting w\u20D7" if miss else "hit",
        }
        for line in data_print(labeled_data):
            print(line)

        if human:
            input(f"done testing: {'miss' if miss else 'hit'}")
        else:
            time.sleep(0.1)

        if miss:
            tmp_vec = Vector(vec.x, vec.y, vec.z)
            tmp_vec.scale(label)
            print(tmp_vec.x, tmp_vec.y, label)
            w3.add(tmp_vec)
            render()
    return misses

for x in range(1000):
    misses = perceptron(False)
    if misses == 0: break
